# `prachathai-67k` Classification Benchmark

In [53]:
import pandas as pd
import numpy as np
from pythainlp import word_tokenize
from ast import literal_eval
from tqdm import tqdm_notebook
from collections import Counter
import re

#viz
from plotnine import *
import matplotlib.pyplot as plt
import seaborn as sns

benchmark_labels = ['การเมือง','สิทธิมนุษยชน','คุณภาพชีวิต','ต่างประเทศ','สังคม',
                  'สิ่งแวดล้อม','เศรษฐกิจ','วัฒนธรรม','แรงงาน','ความมั่นคง','ไอซีที','การศึกษา']

def replace_newline(t):
    return re.sub('[\n]{1,}', ' ', t)

ft_data = 'ft_data/'
ft_train = f'{ft_data}train/'
ft_valid = f'{ft_data}valid/'
ft_test = f'{ft_data}test/'

In [3]:
prachathai = pd.read_csv('prachathai-67k.csv')
print(prachathai.shape)
prachathai.head()

(67889, 5)


,url,date,title,body_text,labels
0,https://prachatai.com/print/42,2004-08-24 14:31,"นักวิจัยหนุน ""แม้ว"" เปิด ""จีเอ็มโอ""",ประชาไท --- 23 ส.ค.2547 นักวิจัยฯ ชี้นโยบายจี...,"['ข่าว', 'สิ่งแวดล้อม']"
1,https://prachatai.com/print/41,2004-08-24 14:22,ภาคประชาชนต้านเปิดเสรีจีเอ็มโอ,ประชาไท- 23 ส.ค.2547 นักวิชาการ ภาคประชาชน จ...,"['ข่าว', 'สิ่งแวดล้อม']"
2,https://prachatai.com/print/43,2004-08-24 15:17,จุฬาฯ ห่วงจีเอ็มโอลามข้าวไทย,นโยบายที่อนุญาตให้ปลูกร่วมกับพืชอื่นได้นั้นถื...,"['ข่าว', 'สิ่งแวดล้อม']"
3,https://prachatai.com/print/45,2004-08-24 15:58,ฟองสบู่การเมืองแตก ทักษิณหมดกึ๋น ชนชั้นกลางหมด...,ประชาไท -- 23 ส.ค. 47 ขาประจำทักษิณ ฟันธง ฟอง...,"['ข่าว', 'การเมือง', 'คณะเศรษฐศาสตร์ มหาวิทยาล..."
4,https://prachatai.com/print/47,2004-08-24 16:10,กอต.เสนอเลิกถนนคลองลาน-อุ้มผาง,ประชาไท-23 ส.ค.47 คณะกรรมการอนุรักษ์ ผืนป่าตะ...,"['ข่าว', 'สิ่งแวดล้อม']"


## Train-validation-test Split

We perform 70/10/20 train-validation-test split. The train-validation vs test split is done randomly and the train vs validation split is done using `MultilabelStratifiedShuffleSplit` as implemented by [iterative-stratification](https://github.com/trent-b/iterative-stratification). We will use the test set for classification benchmark.

In [70]:
all_df = prachathai[['title', 'labels']].copy()
all_df.head()

#use only benchmark labels
for lab in benchmark_labels:
    all_df[lab] = all_df.labels.map(lambda x: 1 if lab in x else 0)
all_df.drop('labels',axis=1,inplace=True)

In [71]:
all_df.head()

,title,การเมือง,สิทธิมนุษยชน,คุณภาพชีวิต,ต่างประเทศ,สังคม,สิ่งแวดล้อม,เศรษฐกิจ,วัฒนธรรม,แรงงาน,ความมั่นคง,ไอซีที,การศึกษา
0,"นักวิจัยหนุน ""แม้ว"" เปิด ""จีเอ็มโอ""",0,0,0,0,0,1,0,0,0,0,0,0
1,ภาคประชาชนต้านเปิดเสรีจีเอ็มโอ,0,0,0,0,0,1,0,0,0,0,0,0
2,จุฬาฯ ห่วงจีเอ็มโอลามข้าวไทย,0,0,0,0,0,1,0,0,0,0,0,0
3,ฟองสบู่การเมืองแตก ทักษิณหมดกึ๋น ชนชั้นกลางหมด...,1,0,0,0,0,0,0,0,0,0,0,0
4,กอต.เสนอเลิกถนนคลองลาน-อุ้มผาง,0,0,0,0,0,1,0,0,0,0,0,0


In [42]:
# #randomly create test set
# idx = np.arange(all_df.shape[0])
# np.random.seed(1412)
# np.random.shuffle(idx)
# train_valid_idx, test_idx = idx[:int(all_df.shape[0] * 0.8)], idx[int(all_df.shape[0] * 0.7):]
# print(len(train_valid_idx),len(test_idx))

# #save test_df
# test_df = all_df.iloc[test_idx,:]
# print(test_df.shape)
# test_df.to_csv('test_df.csv',index=False)

54311 20367
(20367, 13)


In [47]:
# #stratified shuffle split for validation set
# from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit

# train_valid_df = all_df.iloc[train_valid_idx,:]
# ohe_df = train_valid_df.iloc[:,1:]
# msss = MultilabelStratifiedShuffleSplit(n_splits=1, test_size=0.125, random_state=1412)
# train_idx, valid_idx = next(msss.split(ohe_df,ohe_df))
# print(len(train_idx),len(valid_idx))

# #save train and valid_df
# train_df = all_df.iloc[train_idx,:]
# print(train_df.shape)
# train_df.to_csv('train_df.csv',index=False)

# valid_df = all_df.iloc[valid_idx,:]
# print(valid_df.shape)
# valid_df.to_csv('valid_df.csv',index=False)

47522 6789
(47522, 13)
(6789, 13)


In [25]:
train_df = pd.read_csv('train_df.csv')
valid_df = pd.read_csv('valid_df.csv')
test_df = pd.read_csv('test_df.csv')

In [56]:
df_txts = ['train','valid','test']
dfs = [train_df,valid_df,test_df]

for i in range(3):
    df = dfs[i]
    for lab in tqdm_notebook(benchmark_labels):
        ft_lines = []
        for _,row in df.iterrows():
            ft_lab = f'__label__{row[lab]}'
            ft_text = replace_newline(f'{row["body_text"]}')
            ft_line = f'{ft_lab} {ft_text}'
            ft_lines.append(ft_line)

        doc = '\n'.join(ft_lines)
        with open(f'{ft_data}{df_txts[i]}_{lab}.txt','w') as f:
            f.write(doc)
        f.close()

In [72]:
#for fasttext embedding finetuning
ft_lines = []
for _,row in all_df.iterrows():
    ft_lab = '__label__0'
    ft_text = replace_newline(f'{row["body"]}')
    ft_line = f'{ft_lab} {ft_text}'
    ft_lines.append(ft_line)

doc = '\n'.join(ft_lines)
with open(f'{ft_data}df_all.txt','w') as f:
    f.write(doc)
f.close()

In [79]:
!/home/charin/fastText-0.1.0/fasttext skipgram \
-pretrainedVectors 'model/wiki.th.vec' -dim 300 \
-input ft_data/df_all.txt -output 'model/finetuned'

In [81]:
!/home/charin/fastText-0.1.0/fasttext supervised \
-input 'ft_data/train_คุณภาพชีวิต.txt' -output 'model/คุณภาพชีวิต' \
-pretrainedVectors 'model/finetuned.vec' -epoch 10 -dim 300 -wordNgrams 2 

Read 11M words
Number of words:  5773818
Number of labels: 2
Progress: 26.1%  words/sec/thread: 138993  lr: 0.073882  loss: 0.125940  eta: 0h0m ^C
